# SNN Weight & Threshold Balancing - Fully Connected

In [1]:
import torch
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST

import snntorch as snn
import snntorch.functional as SF

from spike_nets import FC_Net, FC_SNN, FC_Count_Net

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
# data alread 0-1 normalised, simply convert to tensor
transform_data = ToTensor()

# Load the data
batch_size = 100
train_dataset = MNIST(root = './mnist/', train = True, download = True, transform=transform_data)
test_dataset = MNIST(root = './mnist/', train = False, download = True, transform=transform_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

## FC ReLU Neural Network

In [3]:
def init_weights(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.uniform_(m.weight, -0.1, 0.1)

fc_net = FC_Net(784, [1200, 1200], 10).to(device)
fc_net.apply(init_weights)
optimiser = torch.optim.Adam(fc_net.parameters())

# optimiser used in the original paper seems to kill the gradients, so we're just going to use adam
# optimiser = torch.optim.SGD(fc_net.parameters(), lr=.01, momentum=0.5)

fc_net.train()

FC_Net(
  (in_layer): Linear(in_features=784, out_features=1200, bias=False)
  (h1_layer): Linear(in_features=1200, out_features=1200, bias=False)
  (h2_layer): Linear(in_features=1200, out_features=10, bias=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (activator): ReLU()
)

In [ ]:
# Training model
num_epochs = 15
for epoch in range(num_epochs):
    # Go trough all samples in train dataset
    for i, (images, labels) in enumerate(train_loader):
        # Get from dataloader and send to device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = fc_net(images)
        # Compute loss
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        # Backward and optimize
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        # Display
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [5]:
# Evaluate model accuracy on test after training
# Set model in eval mode!
fc_net.eval()
# Evaluate
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Get images and labels from test loader
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass and predict class using max
        outputs = fc_net(images)
        _, predicted = torch.max(outputs.data, 1)
        # Check if predicted class matches label
        # and count number of correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
# Compute final accuracy and display
accuracy = correct/total
print(f'Evaluation after training, test accuracy: {accuracy:.4f}')

Evaluation after training, test accuracy: 0.9825


In [10]:
fc_net.save_parameters("params/linear")

## FC Spiking Neural Network

In [3]:
step_count = 100
fc_snn = FC_SNN(784, [1200, 1200], 10, beta=1, threshold=4, steps=step_count, rate=200).to(device)
fc_snn.load_parameters("params/linear")
fc_snn.eval()

FC_SNN(
  (in_layer): Linear(in_features=784, out_features=1200, bias=False)
  (h1_layer): Linear(in_features=1200, out_features=1200, bias=False)
  (h2_layer): Linear(in_features=1200, out_features=10, bias=False)
  (in_active): Leaky()
  (h1_active): Leaky()
  (h2_active): Leaky()
)

In [4]:
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        spk_out, _ = fc_snn(images)

        correct += SF.accuracy_rate(spk_out, labels) * spk_out.size(1)
        total += spk_out.size(1)

accuracy = correct/total
print(f'Evaluation after training, test accuracy: {accuracy:.4f}')

Evaluation after training, test accuracy: 0.9809


## FC Neural Network Model Normalisation

In [3]:
# model based normalisation algorithm
def model_norm(model: torch.nn.Module, include_last: bool = False):
    """
    takes any pytorch module or network and does model normalisation across all layers.
    by default, the last layer is not normalised.
    """
    param_list = [param for param in model.parameters() if param.requires_grad]
    param_geny = iter(param_list)  # reset generator

    param_count = len(param_list)
    layers = param_count if include_last else param_count - 1
    layer_scales = []

    for _ in range(layers):
        neurons = next(param_geny)
        max_pos_in = 0

        if neurons.dim() >= 2:  # if layer is linear/conv

            for neuron in neurons:    
                input_sum = torch.sum(torch.clamp(neuron, min=0)) # sum all positive parameters
                max_pos_in = max(max_pos_in, input_sum.item())

        if max_pos_in > 0:
            neurons.data /= max_pos_in  # without .data this becomes out-of-place for some reason

        layer_scales.append(max_pos_in)

    return layer_scales

In [4]:
# to load and rescale from scratch
fc_mn_snn = FC_SNN(784, [1200, 1200], 10, beta=1).to(device)
fc_mn_snn.load_parameters("params/linear")
fc_mn_snn.eval()

scaling_factors = model_norm(fc_mn_snn, False)
print(scaling_factors)
fc_mn_snn.save_parameters("params/model_norm")

[32.103599548339844, 52.59414291381836]


In [9]:
# to load rescaled weights from file
max_rate = 400
threshold = 0.25
simulation_time = 0.5

fc_mn_snn = FC_SNN(784, [1200, 1200], 10, beta=1, threshold=threshold, steps=int(max_rate * simulation_time), rate=max_rate).to(device)
fc_mn_snn.load_parameters("params/model_norm")
fc_mn_snn.eval()

FC_SNN(
  (in_layer): Linear(in_features=784, out_features=1200, bias=False)
  (h1_layer): Linear(in_features=1200, out_features=1200, bias=False)
  (h2_layer): Linear(in_features=1200, out_features=10, bias=False)
  (in_active): Leaky()
  (h1_active): Leaky()
  (h2_active): Leaky()
)

In [10]:
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        spk_out, _ = fc_mn_snn(images)

        correct += SF.accuracy_rate(spk_out, labels) * spk_out.size(1)
        total += spk_out.size(1)

accuracy = correct/total
print(f'Evaluation after training, test accuracy: {accuracy:.4f}')

Evaluation after training, test accuracy: 0.9803


## FC Neural Network Data Normalisation

In [5]:
# get maximum neuron activations
count_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle = False)
fc_count_net = FC_Count_Net(784, [1200, 1200], 10).to(device)
fc_count_net.load_parameters("params/linear")
fc_count_net.eval()

with torch.no_grad():

    for images, labels in count_loader:

        images = images.to(device)
        labels = labels.to(device)

        _ = fc_count_net(images)

max_activations = [torch.max(fc_count_net.maxin_act), torch.max(fc_count_net.maxh1_act), torch.max(fc_count_net.maxh2_act)]

In [9]:
# algorithm for data normalisation
def data_norm(model: torch.nn.Module, activations: list[torch.Tensor], include_last: bool=True):
    """
    takes a pytorch module or network and does data normalisation.
    requires the list of maximum activations from each layer.
    unlike model normalisation, also normalises the last layer by default.
    """
    param_list = [param for param in model.parameters() if param.requires_grad]
    param_geny = iter(param_list)

    param_count = len(param_list)
    layers = param_count if include_last else param_count - 1
    layer_scales = []

    previous_factor = 1

    for i in range(layers):
        neurons = next(param_geny)
        max_weight = 0

        if neurons.dim() >= 2:

            for neuron in neurons:
                # grab maximum single weight across input connections
                max_weight = max(max_weight, torch.max(neuron))

        if max_weight > 0:
            scale_factor = max(max_weight, activations[i])
            applied_factor = scale_factor / previous_factor

        # rescale all weights wrt applied factor
        neurons.data = neurons / applied_factor # without .data this becomes out-of-place for some reason
        previous_factor = scale_factor
        layer_scales.append(applied_factor)
    
    return layer_scales

In [10]:
# to load and rescale from scratch
fc_dn_snn = FC_SNN(784, [1200, 1200], 10, beta=1).to(device)
fc_dn_snn.load_parameters("params/linear")
fc_dn_snn.eval()

scaling_factors = data_norm(fc_dn_snn, max_activations)
print(scaling_factors)
fc_dn_snn.save_parameters("params/data_norm")

[tensor(8.4872), tensor(2.1156), tensor(4.4810)]


In [17]:
# to load rescaled weights from file
max_rate = 400
threshold = 1
simulation_time = 0.5

fc_dn_snn = FC_SNN(784, [1200, 1200], 10, beta=1, threshold=threshold, steps=int(max_rate * simulation_time), rate=max_rate).to(device)
fc_dn_snn.load_parameters("params/data_norm")
fc_dn_snn.eval()

FC_SNN(
  (in_layer): Linear(in_features=784, out_features=1200, bias=False)
  (h1_layer): Linear(in_features=1200, out_features=1200, bias=False)
  (h2_layer): Linear(in_features=1200, out_features=10, bias=False)
  (in_active): Leaky()
  (h1_active): Leaky()
  (h2_active): Leaky()
)

In [18]:
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        spk_out, _ = fc_dn_snn(images)

        correct += SF.accuracy_rate(spk_out, labels) * spk_out.size(1)
        total += spk_out.size(1)

accuracy = correct/total
print(f'Evaluation after training, test accuracy: {accuracy:.4f}')

Evaluation after training, test accuracy: 0.9815
